# 填充和步幅

## 1. 填充（padding）

由于通常使用小卷积核，因此对于单个卷积，可能只会丢失几个像素。但是应用许多连续卷积层，累积丢失的像素逐渐增多。解决方法即为padding。

如果添加$p_h$行填充（一半在顶部，一半在底部），如果添加$p_w$列填充（一半在左部，一半在右部），输出的形状为：
$$(n_h - k_h+ p_h + 1) \times (n_w - k_w + p_w + 1)$$
假如$k_h$是奇数，在高度两侧填充$p_h/2$行。假如$k_h$是偶数，在顶部填充$\lceil p_h/2 \rceil$, 在底部填充$\lfloor p_h/2 \rfloor$。

卷积核通常为奇数，例如1、3、5、7。选择奇数保持空间维度的同时，在顶部和底部填充相同数量的行，在左侧核右侧填充相同数量的列

**满足如下条件:**
- 卷积核的大小是奇数
- 所有边的填充行和列数相同
- 输出和输入具有相同高度和宽度
可以得到：输出Y[i, j]是通过输入X[i, j]为中心，与卷积核进行互相关计算得到的

In [5]:
'''创建一个高度和宽度为3的二维卷积层
所有侧边填充1个像素
给定高度和宽度为8的输入，则输出的高度和宽度也是8
'''
import torch
from torch import nn

# 定义一个计算卷积层的函数
# 此函数初始化卷积层权重，对输入和输出提高和缩减相应的维数
def comp_conv2d(conv2d, X):
    # 这里的(1, 1)表示批量大小和通道数都是1
    X = X.reshape((1, 1) + X.shape) # X原本的形状是(8, 8)，重塑之后变成(1, 1, 8, 8)，第一个1表示批次大小、第二个1表示通道大小
    Y = conv2d(X) # 执行操作
    return Y.reshape(Y.shape[2:]) # 去掉批次和通道维度，变为形状(8, 8)

conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=(1, 1)) # 该层有一个输入通道和1个输出通道，卷积核大小为3*3.填充设置为1，意味所有输入在所有边上都用零填充1个像素
X = torch.rand(size=(8, 8))
comp_conv2d(conv2d, X). shape

torch.Size([8, 8])

In [6]:
'''当卷积核的高度和宽度不同时，可以填充不同的高度和宽度
使输出和输入具有相同的高度和宽度
如下例中使用高度为5，宽度为3的卷积核，高度和宽度两边分别填充2和1
'''
conv2d = nn.Conv2d(1, 1, kernel_size=(5, 3), padding=(2, 1))
comp_conv2d(conv2d, X).shape


torch.Size([8, 8])

## 2. 步幅（stride）
有时候为了高效计算或是缩减采样次数，卷积窗口可以跳过中间位置，每次滑动多个元素。

当垂直步幅为$s_h$、水平步幅为$s_w$时，输出形状为
$$\left\lfloor \frac{(n_h - k_h + p_h + s_h)}{s_h} \right\rfloor \times \left\lfloor \frac{(n_w - k_w + p_w + s_w)}{s_w} \right\rfloor
$$
如果当我们设置了$p_h=k_h-1$和$p_w=k_w-1$，则输出形状将简化为$$\left\lfloor \frac{(n_h + s_h - 1)}{s_h} \right\rfloor \times \left\lfloor \frac{(n_w + s_w - 1)}{s_w} \right\rfloor$$
其中：n表示输入张量，k表示卷积核，p表示填充，s表示步幅。进一步如果输入高度和宽度可以被垂直和水平步幅整除，则输出形状为
$$(\frac{n_h}{s_h}) \times (\frac{n_w}{s_w})$$

In [7]:
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1, stride=2)
comp_conv2d(conv2d, X).shape

torch.Size([4, 4])

In [8]:
conv2d = nn.Conv2d(1, 1, kernel_size=(3, 5), padding=(0, 1), stride=(3, 4))
comp_conv2d(conv2d, X).shape

torch.Size([2, 2])